In [31]:
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
import os
%matplotlib inline

#dfapril = pd.read_csv("/Users/JordanVani/Documents/NYU/PUI/PUI2016_jmv423/HW4_jmv423/201604-citibike-tripdata.csv")
#dfmay = pd.read_csv("/Users/JordanVani/Documents/NYU/PUI/PUI2016_jmv423/HW4_jmv423/201605-citibike-tripdata.csv")
dfjune = pd.read_csv("/Users/JordanVani/Documents/NYU/PUI/PUI2016_jmv423/HW4_jmv423/201606-citibike-tripdata.csv")
frames = [dfjune]
df = pd.concat(frames)

test if the distribution of ages is the same for 

1) male and female riders

2) day and night riders

use 3 tests: KS, Pearson's, Spearman's. 

For the KS do the test with the entire dataset and with a subset 200 times smaller

Choose a significant threshold for the whole eercise. 

For each test phrase the Null Hypothesis in words.

Describe the return of the scipy function you use in each case.

State the result in terms of rejection of the Null.

# Distribution of Age for Male & Female Riders
#### Question: Are the distribution of ages for male and female citibike riders the same?

In [32]:
df1 = df[['starttime', 'gender', 'birth year']].copy()
df1 = df1.rename(columns = {'birth year' : 'birthyear'})
df1['age'] = 2016 - df1.birthyear
df1['agem'] = np.where(df['gender'] == 1, df1['age'], 'NaN')
df1['agef'] = np.where(df['gender'] == 2, df1['age'], 'NaN')
df1.dropna(0, 'any', inplace = True)
df1.head()

,starttime,gender,birthyear,age,agem,agef
0,6/1/2016 00:00:18,1,1972,44,44.0,NaN
1,6/1/2016 00:00:20,1,1967,49,49.0,NaN
2,6/1/2016 00:00:21,1,1989,27,27.0,NaN
3,6/1/2016 00:00:28,1,1991,25,25.0,NaN
4,6/1/2016 00:00:53,1,1989,27,27.0,NaN


In [ ]:
#pd.cut(male['age'], bins)
#male = df1.query('gender == 1')
#female = df1.query('gender == 2')
agebins = np.arange(10, 99, 5)

fig, (ax_1, ax_2, ax_3) = pl.subplots(nrows=1, ncols=3, figsize=(18,6))
pl.tight_layout(pad=5)
ax_1.hist(df1.agem, bins = agebins)
ax_2.hist(df1.agef, bins = agebins)

### KS Test
Do the test with the entire dataset and with a subset 200x smaller.

$H_o$: There is no difference between the age distributions of male and female citibike riders (significance at p=0.05).

$H_a$: There is a difference between the age distributions of male and female citibike riders (significance at p=0.05).

### Pearson's Test

$H_o$: There is no difference between the age distributions of male and female citibike riders (significance at p=0.05).

$H_a$: There is a difference between the age distributions of male and female citibike riders (significance at p=0.05).

### Spearman's Test

$H_o$: There is no difference between the age distributions of male and female citibike riders (significance at p=0.05).

$H_a$: There is a difference between the age distributions of male and female citibike riders (significance at p=0.05).